<a href="https://colab.research.google.com/github/SSDivyaRavali/CDS/blob/main/Module5/Copy_of_M5_SNB_MiniProject_4_End_to_End_Analytics_Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification Program in Computational Data Science
## A program by IISc and TalentSprint
### Mini-Project: End-to-end analytics application using Pyspark

**DISCLAIMER:** THIS NOTEBOOK IS PROVIDED ONLY AS A REFERENCE SOLUTION NOTEBOOK FOR THE MINI-PROJECT. THERE MAY BE OTHER POSSIBLE APPROACHES/METHODS TO ACHIEVE THE SAME RESULTS.

## Problem Statement

Perform sentiment classification by analyzing the tweets data with Pyspark

## Learning Objectives

At the end of the mini-project, you will be able to :

* analyze the text data using pyspark
* derive the insights and visualize the data
* implement feature extraction and classify the data
* train the classification model and deploy

### Dataset

The dataset chosen for this mini-project is **[Twitter US Airline Sentiment](https://data.world/socialmediadata/twitter-us-airline-sentiment)**. It is a record of tweets about airlines in the US. It was created by scraping Twitter data from February 2015. Contributors were asked to first classify positive, negative, and neutral tweets, followed by categorizing negative reasons (such as "late flight" or "rude service").  Along with other information, it contains ID of a Tweet, the sentiment of a tweet ( neutral, negative and positive), reason for a negative tweet, name of airline and text of a tweet.

## Information

The airline industry is a very competitive market that has grown rapidly in the past 2 decades. Airline companies resort to traditional customer feedback forms which in turn are very tedious and time consuming. This is where Twitter data serves as a good source to gather customer feedback tweets and perform sentiment analysis. This dataset comprises of tweets for 6 major US Airlines and a multi-class classification can be performed to categorize the sentiment (neutral, negative, positive). For this mini-project we will start with pre-processing techniques to clean the tweets and then represent these tweets as vectors. A classification algorithm will be used to predict the sentiment for unseen tweets data. The end-to-end analytics will be performed using Pyspark.

## Grading = 10 Points

#### Install Pyspark

In [1]:
#@title Install packages and download the dataset
!pip -qq install pyspark
!pip -qq install handyspark
!wget -qq https://cdn.iisc.talentsprint.com/CDS/MiniProjects/US_Airline_Tweets.csv
print("Packages installed successfully and dataset downloaded!!")

     |████████████████████████████████| 281.4 MB 24 kB/s 
     |████████████████████████████████| 198 kB 53.1 MB/s 
Packages installed successfully and dataset downloaded!!


#### Import required packages

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from handyspark import *
#import seaborn as sns
from matplotlib import pyplot as plt
import re
import string
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.classification import NaiveBayes
from pyspark.sql.types import ArrayType, StringType
import warnings
warnings.filterwarnings("ignore")

In [3]:
# NLTK imports
import nltk
nltk.download('punkt')
# Download stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### Data Loading

#### Start a Spark Session

Spark session is a combined entry point of a Spark application, which came into implementation from Spark 2.0. It provides a way to interact with various Spark functionalities, with a lesser number of constructs.

In [ ]:
spark = SparkSession.builder.appName('TwitterSentiment').getOrCreate()

#### Load the data and infer the schema

To load the dataset use the `read.csv` with `inferSchema` and `header` as parameters.

In [ ]:
dataset = spark.read.csv("/content/US_Airline_Tweets.csv",inferSchema=True,header=True)

In [ ]:
dataset.show(1)

In [ ]:
dataset.count()

### EDA & Visualization ( 2 points)

#### Visualize the horizontal barplot of airline_sentiment (positive, negative, neutral)

Convert the data to handyspark and remove the other records from the column except 3 values mentioned above and plot the graph

In [ ]:
handyDf = dataset.toHandy()

handyDf = handyDf.filter(handyDf["airline_sentiment"].isin(["neutral","positive","negative"]))
sentiments = handyDf.cols['airline_sentiment'][:].value_counts() #.isin(['neutral','positive','negative'])]
sentiments.plot.barh()

In [ ]:
handyDf.count()

#### Plot the number of tweets received for each airline

In [ ]:
handyDf.cols['airline'][:].value_counts().plot.bar()

#### Visualize a stacked barchart of 6 US airlines and 3 sentiments on each bar

* Display the count corresponding to each sentiment in each bar. [hint](https://priteshbgohil.medium.com/stacked-bar-chart-in-python-ddc0781f7d5f)

In [ ]:
grouped = handyDf.groupby('airline','airline_sentiment').agg(count('airline_sentiment'))
neutral = grouped.filter(grouped['airline_sentiment']=='neutral').cols['count(airline_sentiment)'][:]
positive = grouped.filter(grouped['airline_sentiment']=='positive').cols['count(airline_sentiment)'][:]
negative = grouped.filter(grouped['airline_sentiment']=='negative').cols['count(airline_sentiment)'][:]

In [ ]:
airlines = list(set(handyDf.cols['airline'][:]))
plt.figure(figsize=(10,7))

ax1 = plt.bar(airlines,neutral.values,color='y',label='neutral')
ax2 = plt.bar(airlines,positive.values,bottom=neutral.values,color='b',label='positive')
ax3 = plt.bar(airlines,negative.values,bottom=neutral.values+positive.values,color='r',label='negative')
for r1, r2, r3 in zip(ax1, ax2, ax3):
    h1 = r1.get_height()
    h2 = r2.get_height()
    h3 = r3.get_height()
    plt.text(r1.get_x() + r1.get_width() / 2., h1 / 2., "%d" % h1, ha="center", va="center", color="white", fontsize=16, fontweight="bold")
    plt.text(r2.get_x() + r2.get_width() / 2., h1 + h2 / 2., "%d" % h2, ha="center", va="center", color="white", fontsize=16, fontweight="bold")
    plt.text(r3.get_x() + r3.get_width() / 2., h1 + h2 + h3 / 2., "%d" % h3, ha="center", va="center", color="white", fontsize=16, fontweight="bold")
plt.legend()#['neutral','positive','negative'])
plt.show()

#### Visualize the horizontal barplot of negative reasons

In [ ]:
handyDf1 = handyDf.filter(~handyDf["negativereason"].isNull())
negativereason = handyDf1.cols['negativereason'][:].value_counts() #.isin(['neutral','positive','negative'])]
negativereason.plot.barh()

### Pre-processing (3 points)

#### Check the null values and drop the records where the text value is null

In [ ]:
# check the null values in text column
dataset.filter(dataset.text.isNull()).count()

In [ ]:
# filter out and remove null values from text column
dataset_filtered = dataset.filter(dataset.text.isNotNull())

In [ ]:
# verify the null count
dataset.count(), dataset_filtered.count()

#### Fill the null values with 0 in all the columns except the target

The target should not be empty. Ensure that all features are integer type, convert if needed.

In [ ]:
fillNull = udf(lambda x: 0 if x == None else x)
dataset_filtered = dataset_filtered.withColumn('negativereason_confidence',fillNull(dataset_filtered['negativereason_confidence']).astype('int'))
dataset_filtered = dataset_filtered.withColumn('airline_sentiment_confidence',fillNull(dataset_filtered['airline_sentiment_confidence']).astype('int'))

#### Preprocessing and cleaning the tweets

* Convert the text to lower case
* Remove usernames, hashtags and links from the text (tweets)

In [ ]:
puncts = "!#$%&\()*+,-./:;<=>?@[\\]^_`{|}~"
def words_process(text):
    text = text.lower() # lowercase
    text.replace(r'http?://[^\s<>"]+|www\.[^\s<>"]+', '') # Removing hyperlinks from all the tweets
    text.replace('\d+', '') # Removing numbers from all the tweets
    text = " ".join([i for i in text.split() if i.find("@")== -1]) # removing usernames
    text = text.replace('#','') # Removing hashtags, including the text, from all the tweets
    text = re.sub(r'[^a-zA-Z0-9 ]',r'',text)
    return text

words = udf(words_process)
dataset_filtered = dataset_filtered.withColumn('text_processed',words(dataset_filtered['text']))
dataset_filtered.show()

#### Tokenize the text sentence into words using nltk sentence tokenizer

In [ ]:
word_udf = udf(lambda x: word_tokenize(x), ArrayType(StringType()))
dataset_filtered = dataset_filtered.withColumn("wordss", word_udf("text_processed"))
dataset_filtered.show(5)

#### Remove the stopwords from tokenized words

In [ ]:
stop_words = set(stopwords.words('english'))
print(stop_words)

In [ ]:
punct_udf1 = udf(lambda x: [w for w in x if not w in stop_words])
dataset_filtered = dataset_filtered.withColumn("wordss", punct_udf1("wordss"))

array_udf = udf(lambda x: x, ArrayType(StringType()))
dataset_filtered = dataset_filtered.withColumn("wordss", array_udf("wordss"))

#### Apply Lemmatization to the words

In [ ]:
def lemmatize(text_arr):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in text_arr]
lem = udf(lemmatize)
dataset_filtered = dataset_filtered.withColumn("wordss", lem("wordss"))

array_udf = udf(lambda x: x, ArrayType(StringType()))
dataset_filtered = dataset_filtered.withColumn("wordss", array_udf("wordss"))

In [ ]:
dataset_filtered.show(5)

### Feature Extraction (3 points)

Create the useful features from the text column to train the model

For example:
* Length of the tweet 
* No. of hashtags in the tweet starting with '#'
* No. of mentions in the tweet starting with '@'

Hint: create a new column for each of the above features

* create a column "Length of tweet" using `udf` function

In [ ]:
# CODE HERE
length = udf(lambda x: len(x))

dataset_filtered = dataset_filtered.withColumn('tweet_length', length(dataset_filtered['text_processed']).astype('int'))
dataset_filtered.show()

* Create a new column "No.of Hashtags" in each tweet

In [ ]:
num_hashtags = udf(lambda x : len(re.compile(r"#(\w+)").findall(x)))

dataset_filtered = dataset_filtered.withColumn('num_hashtags', num_hashtags(dataset_filtered['text']).astype('int'))
dataset_filtered.show()

* Create a new column "No.of Mentions" in each tweet

In [ ]:
num_mentions = udf(lambda x : len(re.compile(r"@(\w+)").findall(x)))

dataset_filtered = dataset_filtered.withColumn('num_mentions', num_mentions(dataset_filtered['text']).astype('int'))
dataset_filtered.show()

* Create a new column "Punctuation Count" in each tweet

In [ ]:
def punctCount1(text):
    sum = 0
    for i in text:
        if i in "!$%&'()*+,-./:;<=>?[\]^_`{|}~":
            sum +=1
    return sum

punctCount = udf(punctCount1)
dataset_filtered = dataset_filtered.withColumn('PunctCount',punctCount(dataset_filtered['text']).astype('int'))
dataset_filtered.select('PunctCount').show()

* Create a new column "Type of Punctuations" used in each tweet

In [ ]:
def types_punctuation(text):
    return len(set([i for i in text if i in "!$%&'()*+,-./:;<=>?[\]^_`{|}~"]))

typePunct = udf(types_punctuation)
dataset_filtered = dataset_filtered.withColumn('typePunct',typePunct(dataset_filtered['text']).astype('int'))
dataset_filtered.show(10)

#### Get the features by applying CountVectorizer
CountVectorizer converts the list of tokens to vectors of token counts. See the [documentation](https://spark.apache.org/docs/latest/ml-features.html#countvectorizer) for details.

In [ ]:
count = CountVectorizer(inputCol="wordss", outputCol="rawFeatures")
count_model = count.fit(dataset_filtered)
featurizedData = count_model.transform(dataset_filtered)
featurizedData.show(truncate=False)

#### Encode the labels

Using the `udf` function encode the string values of *airline_sentiment* to integers.

In [ ]:
def LabelEncoder(x):
    if x == 'positive':
        return 0
    elif x == 'negative':
        return 1
    return 2
encoder = udf(LabelEncoder)
featurizedData = featurizedData.withColumn('label', encoder(featurizedData['airline_sentiment']).astype('int'))
featurizedData.show()

### Train the classifier the evaluate (1 point)

Create vector assembler with the selected features to train the model

In [ ]:
featureassembler = VectorAssembler(inputCols=['rawFeatures','airline_sentiment_confidence','negativereason_confidence',
                                              'tweet_length','num_hashtags','num_mentions','retweet_count','PunctCount','typePunct'] 
                                   ,outputCol='features')
features = featureassembler.transform(featurizedData)
features.select('features').show(truncate=False)

#### Arrange features and label and split them into train and test.

In [ ]:
featuresAndLabels = features.withColumn('labels',featurizedData['label'])
final = featuresAndLabels.select('features','labels')
final.show()

In [ ]:
train_data,test_data = final.randomSplit([0.75,0.25])

#### Train the model with train data and make predictions on the test data

For classification of text data, implement NaiveBayes classifier. It is a probabilistic machine learning model.

For more information about **NaiveBayes Classifier**, click [here](https://spark.apache.org/docs/latest/ml-classification-regression.html#naive-bayes)

In [ ]:
nb = NaiveBayes(featuresCol='features', labelCol='labels')
model = nb.fit(train_data)

In [ ]:
# get the predictions
pred_results = model.transform(test_data)
pred_results.select('prediction').show(10)

#### Evaluate the model and find the accuracy

Compare the labels and predictions and find how many are correct.

To find the accuracy, get the count of correct predictions from test data and divide by the total amount of test dataset.

**Hint:** convert the predictions dataframe to pandas and compare with labels

In [ ]:
# converting to pandas df
preds = pred_results.select('labels','prediction').toPandas()
preds

In [ ]:
# comparing labels and predictions of test data
(preds['labels'] == preds['prediction']).sum() / len(preds)

As a next phase of implementation, we will implement logistic regression model.


### Implementing logistic regression.


For more information about logistic regression, click upon this [button](https://spark.apache.org/docs/2.3.1/api/python/pyspark.ml.html#pyspark.ml.classification.LogisticRegression) to proceed to spark's documentation of logistic regression.

In [ ]:
# Importing the necessary libraries
from pyspark.ml.classification import LogisticRegression
import numpy
from numpy import allclose
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [ ]:
# Creating features column through vector assembler
assembler = VectorAssembler(inputCols=['rawFeatures','airline_sentiment_confidence','negativereason_confidence',
                                              'tweet_length','num_hashtags','num_mentions','retweet_count','PunctCount','typePunct'] 
                                   ,outputCol='features')
df = assembler.transform(featurizedData)
df.show()

In [ ]:
# Creating an object of stringindexer
label_stringIdx = StringIndexer(inputCol = 'label', outputCol = 'labelIndex')

In [ ]:
# Creating output column for labels or target variable.
df = label_stringIdx.fit(df).transform(df)
df.show()

In [ ]:
# Splitting the data into train and test
train, test = df.randomSplit([0.7, 0.3], seed = 2018)

In [ ]:
# Creating object of logistic regression
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)

In [ ]:
lrModel = lr.fit(train)

### Making predictions and evaluating the model

In [ ]:
predictions = lrModel.transform(test)

In [ ]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")

In [ ]:
accuracy = evaluator.evaluate(predictions)

In [ ]:
print("Accuracy = %s" % (accuracy))

### Deployment (1 point)


Deployment is done for both the models:
1. Naive bayes
2. Logistic regression


Let's integrate all the above code snippets in app.py and run it with **Streamlit**.

From the start (data loading step), place every code in app.py including data preprocessing, feature extraction and model training.

**Similar procedure will be followed for logistic regression model deployment**
* In this case, the name of the app file created will be app2.py

* implement the `predict_users_Input()` function which takes one tweet input from user and returns the prediction using the trained model.

* use the same preprocessing techniques and features extraction used for train data on user input.

* user input can be captured from the textbox from **Streamlit** app. Action is triggered when predict button is clicked and user input is classified using `predict_users_Input()` function.

For More information about Streamlit, click [here](https://docs.streamlit.io/en/stable/)

In [ ]:
!pip install -qq streamlit

In [ ]:
%%writefile app.py
import streamlit as st
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import re
import string
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.classification import NaiveBayes
from pyspark.sql.types import ArrayType, StringType
from handyspark import *

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

st.write("Creating a spark session :heavy_check_mark:")
spark = SparkSession.builder.appName('TwitterSentiment').getOrCreate()
dataset = spark.read.csv("/content/US_Airline_Tweets.csv",inferSchema=True,header=True)
dataset_filtered = dataset.filter(dataset.text.isNotNull())
fillNull = udf(lambda x: 0 if x == None else x)
dataset_filtered = dataset_filtered.withColumn('negativereason_confidence',fillNull(dataset_filtered['negativereason_confidence']).astype('int'))
dataset_filtered = dataset_filtered.withColumn('airline_sentiment_confidence',fillNull(dataset_filtered['airline_sentiment_confidence']).astype('int'))

hdf = dataset.toHandy()

def words_process(text):
    text = text.lower() # lowercase
    text.replace(r'http?://[^\s<>"]+|www\.[^\s<>"]+', '') # Removing hyperlinks from all the tweets
    text.replace('\d+', '') # Removing numbers from all the tweets
    text = " ".join([i for i in text.split() if i.find("@")== -1]) # removing usernames
    text = text.replace('#','') # Removing hashtags, including the text, from all the tweets
    return text

words = udf(words_process)
dataset_filtered = dataset_filtered.withColumn('text_processed',words(dataset_filtered['text']))

word_udf = udf(lambda x: word_tokenize(x), ArrayType(StringType()))
dataset_filtered = dataset_filtered.withColumn("wordss", word_udf("text_processed"))
stop_words = set(stopwords.words('english'))
punct_udf1 = udf(lambda x: [w for w in x if not w in stop_words])
dataset_filtered = dataset_filtered.withColumn("wordss", punct_udf1("wordss"))

array_udf = udf(lambda x: x, ArrayType(StringType()))
dataset_filtered = dataset_filtered.withColumn("wordss", array_udf("wordss"))

def lemmatize(text_arr):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in text_arr]
lem = udf(lemmatize)
dataset_filtered = dataset_filtered.withColumn("wordss", lem("wordss"))
st.write("Preprocessing Done! :heavy_check_mark:")

array_udf = udf(lambda x: x, ArrayType(StringType()))
dataset_filtered = dataset_filtered.withColumn("wordss", array_udf("wordss"))

length = udf(lambda x: len(x))
dataset_filtered = dataset_filtered.withColumn('tweet_length', length(dataset_filtered['text']).astype('int'))
num_hashtags = udf(lambda x : len(re.compile(r"#(\w+)").findall(x)) if len(re.compile(r"#(\w+)").findall(x)) > 0 else 0)

dataset_filtered = dataset_filtered.withColumn('num_hashtags', num_hashtags(dataset_filtered['text']).astype('int'))
num_mentions = udf(lambda x : len(re.compile(r"@(\w+)").findall(x)) if len(re.compile(r"@(\w+)").findall(x)) > 0 else 0)

dataset_filtered = dataset_filtered.withColumn('num_mentions', num_mentions(dataset_filtered['text']).astype('int'))
def punctCount1(text):
    sum = 0
    for i in text:
        if i in "!$%&'()*+,-./:;<=>?[\]^_`{|}~":
            sum +=1
    return sum

punctCount = udf(punctCount1)

dataset_filtered = dataset_filtered.withColumn('PunctCount',punctCount(dataset_filtered['text']).astype('int'))
def types_punctuation(text):
    return len(set([i for i in text if i in "!$%&'()*+,-./:;<=>?[\]^_`{|}~"]))

typePunct = udf(types_punctuation)
dataset_filtered = dataset_filtered.withColumn('typePunct',typePunct(dataset_filtered['text']).astype('int'))

st.write("Ongoing feature extraction!! :heavy_check_mark:")
count = CountVectorizer(inputCol="wordss", outputCol="rawFeatures")
count_model = count.fit(dataset_filtered)
featurizedData = count_model.transform(dataset_filtered)

def LabelEncoder(x):
    if x == 'positive':
        return 0
    elif x == 'negative':
        return 1
    return 2
encoder = udf(LabelEncoder)
featurizedData = featurizedData.withColumn('label', encoder(featurizedData['airline_sentiment']).astype('int'))

featureassembler = VectorAssembler(inputCols=['rawFeatures','airline_sentiment_confidence','negativereason_confidence',
                                              'tweet_length','num_hashtags','num_mentions','retweet_count','PunctCount','typePunct'] 
                                   ,outputCol='features')
features = featureassembler.transform(featurizedData)

final = features.withColumn('labels',featurizedData['label'])
train_data,test_data = final.randomSplit([0.75,0.25])
nb = NaiveBayes(featuresCol='features', labelCol='labels')
st.write("Training the naive bayes model :heavy_check_mark:")

model = nb.fit(train_data)

def predict_users_Input(user_input):
  df1 = spark.createDataFrame([ (1, user_input)],['Id', 'UserTweet'])

  df1 = df1.withColumn('UserTweet',words(df1['UserTweet']))
  df1 = df1.withColumn("wordss", word_udf("UserTweet"))
  df1 = df1.withColumn("wordss", punct_udf1("wordss"))
  df1 = df1.withColumn("wordss", array_udf("wordss"))
  df1 = df1.withColumn("wordss", lem("wordss"))
  df1 = df1.withColumn("wordss", array_udf("wordss"))

  df1 = df1.withColumn('tweet_length', length(df1['UserTweet']).astype('int'))
  df1 = df1.withColumn('num_hashtags', num_hashtags(df1['UserTweet']).astype('int'))
  df1 = df1.withColumn('num_mentions', num_mentions(df1['UserTweet']).astype('int'))
  df1 = df1.withColumn('PunctCount',punctCount(df1['UserTweet']).astype('int'))
  df1 = df1.withColumn('typePunct',typePunct(df1['UserTweet']).astype('int'))
  make  = udf(lambda x : 0)
  df1 = df1.withColumn('negativereason_confidence',make(df1['UserTweet']).astype('int'))
  df1 = df1.withColumn('airline_sentiment_confidence',make(df1['UserTweet']).astype('int'))
  df1 = df1.withColumn('retweet_count',make(df1['UserTweet']).astype('int'))
  df1_featured = count_model.transform(df1)
  test_features = featureassembler.transform(df1_featured)
  test_predict = model.transform(test_features)
  df_res = test_predict.select('prediction').toPandas()
  return df_res

def decode(label):
  if label == 0:
    return "Positive tweet!"
  elif label == 1:
    return "Negative Tweet!"
  return "Neutral tweet"

user_input = st.text_input("Enter the text input","Your tweet here ")
if st.button('predict'):
    result = predict_users_Input(user_input)
    st.write(decode(result.prediction.values[0]))

#### Writing aap2.py file for logistic regression model

In [ ]:
%%writefile app2.py
import streamlit as st
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import re
import string
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.types import ArrayType, StringType
from handyspark import *
import numpy
from numpy import allclose
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StringIndexer

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import os
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.ml.feature import CountVectorizerModel
st.write("Creating a spark session :heavy_check_mark:")
spark = SparkSession.builder.appName('TwitterSentiment').getOrCreate()


def words_process(text):
    text = text.lower() # lowercase
    text.replace(r'http?://[^\s<>"]+|www\.[^\s<>"]+', '') # Removing hyperlinks from all the tweets
    text.replace('\d+', '') # Removing numbers from all the tweets
    text = " ".join([i for i in text.split() if i.find("@")== -1]) # removing usernames
    text = text.replace('#','') # Removing hashtags, including the text, from all the tweets
    return text

def punctCount1(text):
    sum = 0
    for i in text:
        if i in "!$%&'()*+,-./:;<=>?[\]^_`{|}~":
            sum +=1
    return sum


def LabelEncoder(x):
    if x == 'positive':
        return 0
    elif x == 'negative':
        return 1
    return 2

def lemmatize(text_arr):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in text_arr]

def types_punctuation(text):
    return len(set([i for i in text if i in "!$%&'()*+,-./:;<=>?[\]^_`{|}~"]))



words = udf(words_process)
word_udf = udf(lambda x: word_tokenize(x), ArrayType(StringType()))
stop_words = set(stopwords.words('english'))
punct_udf1 = udf(lambda x: [w for w in x if not w in stop_words])
array_udf = udf(lambda x: x, ArrayType(StringType()))
punctCount = udf(punctCount1)
encoder = udf(LabelEncoder)
lem = udf(lemmatize)
length = udf(lambda x: len(x))
array_udf = udf(lambda x: x, ArrayType(StringType()))
typePunct = udf(types_punctuation)
num_hashtags = udf(lambda x : len(re.compile(r"#(\w+)").findall(x)) if len(re.compile(r"#(\w+)").findall(x)) > 0 else 0)
num_mentions = udf(lambda x : len(re.compile(r"@(\w+)").findall(x)) if len(re.compile(r"@(\w+)").findall(x)) > 0 else 0)
fillNull = udf(lambda x: 0 if x == None else x)
assembler = VectorAssembler(inputCols=['rawFeatures','airline_sentiment_confidence','negativereason_confidence',
                                                'tweet_length','num_hashtags','num_mentions','retweet_count','PunctCount','typePunct'] 
                                    ,outputCol='features')



def get_model():
  if os.path.exists("lrmodel"):
      return LogisticRegressionModel.load('lrmodel'),CountVectorizerModel.load('countmodel')
  dataset = spark.read.csv("/content/US_Airline_Tweets.csv",inferSchema=True,header=True)
  hdf = dataset.toHandy()
  dataset_filtered = dataset.filter(dataset.text.isNotNull())
  dataset_filtered = dataset_filtered.withColumn('negativereason_confidence',fillNull(dataset_filtered['negativereason_confidence']).astype('int'))
  dataset_filtered = dataset_filtered.withColumn('airline_sentiment_confidence',fillNull(dataset_filtered['airline_sentiment_confidence']).astype('int'))
  dataset_filtered = dataset_filtered.withColumn('text_processed',words(dataset_filtered['text']))
  dataset_filtered = dataset_filtered.withColumn("wordss", word_udf("text_processed"))
  dataset_filtered = dataset_filtered.withColumn("wordss", punct_udf1("wordss"))
  dataset_filtered = dataset_filtered.withColumn("wordss", array_udf("wordss"))

  dataset_filtered = dataset_filtered.withColumn("wordss", lem("wordss"))
  st.write("Preprocessing Done! :heavy_check_mark:")


  dataset_filtered = dataset_filtered.withColumn("wordss", array_udf("wordss"))


  dataset_filtered = dataset_filtered.withColumn('tweet_length', length(dataset_filtered['text']).astype('int'))

  dataset_filtered = dataset_filtered.withColumn('num_hashtags', num_hashtags(dataset_filtered['text']).astype('int'))

  dataset_filtered = dataset_filtered.withColumn('num_mentions', num_mentions(dataset_filtered['text']).astype('int'))

  dataset_filtered = dataset_filtered.withColumn('PunctCount',punctCount(dataset_filtered['text']).astype('int'))
  dataset_filtered = dataset_filtered.withColumn('typePunct',typePunct(dataset_filtered['text']).astype('int'))

  st.write("Ongoing feature extraction!! :heavy_check_mark:")
  count = CountVectorizer(inputCol="wordss", outputCol="rawFeatures")
  count_model = count.fit(dataset_filtered)
  count_model.setInputCol("wordss")
  count_model.save('countmodel')
  featurizedData = count_model.transform(dataset_filtered)


  featurizedData = featurizedData.withColumn('label', encoder(featurizedData['airline_sentiment']).astype('int'))

  df = assembler.transform(featurizedData)

  train, test = df.randomSplit([0.7, 0.3], seed = 2018)
  lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)

  st.write("Training the logistic regression model :heavy_check_mark:")

  lrModel = lr.fit(train)
  lrModel.save('lrmodel')
  return lrModel,count_model

lrModel,count_model = get_model()

def predict_users_Input(user_input):
  df1 = spark.createDataFrame([ (1, user_input)],['Id', 'UserTweet'])

  df1 = df1.withColumn('UserTweet',words(df1['UserTweet']))
  df1 = df1.withColumn("wordss", word_udf("UserTweet"))
  df1 = df1.withColumn("wordss", punct_udf1("wordss"))
  df1 = df1.withColumn("wordss", array_udf("wordss"))
  df1 = df1.withColumn("wordss", lem("wordss"))
  df1 = df1.withColumn("wordss", array_udf("wordss"))

  df1 = df1.withColumn('tweet_length', length(df1['UserTweet']).astype('int'))
  df1 = df1.withColumn('num_hashtags', num_hashtags(df1['UserTweet']).astype('int'))
  df1 = df1.withColumn('num_mentions', num_mentions(df1['UserTweet']).astype('int'))
  df1 = df1.withColumn('PunctCount',punctCount(df1['UserTweet']).astype('int'))
  df1 = df1.withColumn('typePunct',typePunct(df1['UserTweet']).astype('int'))
  make  = udf(lambda x : 0)
  df1 = df1.withColumn('negativereason_confidence',make(df1['UserTweet']).astype('int'))
  df1 = df1.withColumn('airline_sentiment_confidence',make(df1['UserTweet']).astype('int'))
  df1 = df1.withColumn('retweet_count',make(df1['UserTweet']).astype('int'))
  df1_featured = count_model.transform(df1)
  predictions = lrModel.transform(assembler.transform(df1_featured))
  df_res = predictions.select('prediction').toPandas()
  return df_res

def decode(label):
  if label == 0:
    return "Positive tweet!"
  elif label == 1:
    return "Negative Tweet!"
  return "Neutral tweet"

user_input = st.text_input("Enter the text input","Your tweet here ")
if st.button('predict'):
    result = predict_users_Input(user_input)
    st.write(decode(result.prediction.values[0]))

In [ ]:
# To check if the app.py file has written in the current colab sandbox
!ls

#### Ngrok

Ngrok is a powerful solution for providing secure tunnels from our local system to the public. Here, we first need to signup to [ngrok.com](https://ngrok.com/) and create a free account.

**Acount creation is mandatory to get an authentication token**

For more information to run Streamlit apps from Colab refer [here](https://medium.com/@jcharistech/how-to-run-streamlit-apps-from-colab-29b969a1bdfc).

In [ ]:
# Install pyngrok, version 4.1.1 is mandatory
!pip -qq install pyngrok==4.1.1

In [ ]:
# Import ngrok
from pyngrok import ngrok

#### Get Your Authentication Tokens

To use ngrok, you will need an authentication token which can be found on the dashboard of your ngrok [account](https://dashboard.ngrok.com/get-started/setup).

In [ ]:
#!ngrok authtoken 'paste your authtoken here'

!ngrok authtoken 24AzeYQn6rm5KDAA3AH5mh84WpU_3YtjyKe2y3y1F6QbBdijV

#### Start the app

Now that we have written our app, we can now start our app like we would have done if we were running it locally. But the caveat is to run it in the background so that if the cell finish running, our app will continue to run as a background process behind.

### Web URL will be generated only for one app at a time. 
* If app.py for naive bayes is to be run then(#!streamlit run app2.py&>/dev/null&)the streamlit command used for starting app2.py should be commented out.

### Similarly 

* If app2.py for logistic regression is to be run then(#!streamlit run app.py&>/dev/null&)the streamlit command used for starting app.py should be commented out.

Starting app.py file for naive bayes




In [ ]:
# Start app
!streamlit run app.py&>/dev/null&

Starting app2.py file for logistic regression


In [ ]:
# Start app
!streamlit run app2.py&>/dev/null&

This will start streamlit on the normal default port of 8501 which we will use for the next section in pyngrok.

#### Create A Secure Tunnel Using Pyngrok

As we learnt earlier ngrok allows us to create a secure tunnel for accessing our local apps and webhooks. We will be using pyngrok which acts as a python wrapper around ngrok.

To create our tunnel we will be using pyngrok and passing in the port from streamlit (ie 8501) .

After you execute the code below you will get a web app link where you could perform the sentiment prediction task.

In [ ]:
# Generate url
public_url = ngrok.connect(port='8501')
public_url

This will generate a public URL that your app will be running on.


Refer the screenshot below.
![img](https://cdn.iisc.talentsprint.com/CDS/MiniProjects/sentiment_analysis_streamlit_button.JPG)

#### Shutting Down Your App

In case you want to shut down your app, you will have to first kill the streamlit process if it is running on and then shutdown the ngrok from python.

In [ ]:
# Check if streamlit is running
!pgrep streamlit

In [ ]:
# Kill the process by specifying its number from above
!kill 2513

In [ ]:
# Shutdown ngrok from python
ngrok.kill()

In [ ]:
# Python library to run streamlit, flask, fastapi, etc on Google Colab
#!pip install -qq colab-everything
#from colab_everything import ColabStreamlit
#ColabStreamlit('/content/app.py')